In [ ]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd

# Đường dẫn đến tập tin CSV
file_path = 'C:\\Users\\Dell\\OneDrive\\Desktop\\BD_DQ\\data_dotquy.csv'
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

In [ ]:
print(df)

In [ ]:
bins = [0, 18, 35, 60, 100]
labels = ['Trẻ em', 'Thanh niên', 'Trung niên', 'Người già']
df['nhóm tuổi'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
df

In [ ]:
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df

In [ ]:
print("Vinh Hien")